In [124]:
import imcut.pycut
import numpy as np
import random
import matplotlib.pyplot as plt
import scipy
import scipy.ndimage
from skimage import morphology


In [125]:
sliceId = 1
A = (4,10)
B = [sliceId]
C = tuple(B + list(A))
print(A)
print(B)
print(C)
D = [(4, 1), (6, 6), (3, 5)]
E = [ tuple([sliceId] + list(tupleElem)) for  tupleElem in D]
print(E)
F = np.zeros([2, 10, 10], dtype=np.uint8)
for coord in E : F[coord] = 1
#print(F)

(4, 10)
[1]
(1, 4, 10)
[(1, 4, 1), (1, 6, 6), (1, 3, 5)]


In [126]:
def disk(n):
    struct = np.zeros((2 * n + 1, 2 * n + 1))
    x, y = np.indices((2 * n + 1, 2 * n + 1))
    mask = (x - n)**2 + (y - n)**2 <= n**2
    struct[mask] = 1
    return struct.astype(np.bool)
# G = np.zeros([16, 16], dtype=np.uint8)
# G[5:13,4:7] = 1
# print('G')
# print(G)
# dilated_G = scipy.ndimage.binary_dilation(G,structure=disk(2)).astype(G.dtype)
# print('dilated_G')
# print(dilated_G)

In [127]:
def chooseScribbleFromMissedFGOrWrongCBG(misclassifiedRegion, fractionDivider, dilation_diam):
    """
        misclassifiedRegion : int8 binary volume of fgMissed or bgWrongC with slice as first dimension 
        fractionDivider : positive integer by which number of one pixels in a slice will be divide
                to decide what fraction of them will be chosen. 
                If fractionDivider=1, all of them get chosen
        dilation_diam: dimeter of disk : 1,2,3: disk diameter of  scribble
    """
    resultBinary = np.zeros_like(misclassifiedRegion)
    numSlices = misclassifiedRegion.shape[0]
    for sliceId in range(numSlices):
        onePixelsThisSlice = np.where(misclassifiedRegion[sliceId,...]==1)    
        onePixelCoordsThisSlice = list(zip(onePixelsThisSlice[0], onePixelsThisSlice[1]))
        #print(missedFGResultCoordsThisSlice)
        numOnePixelCoordsThisSlice = len(onePixelCoordsThisSlice)
        numScribblesFromThisSlice = numOnePixelCoordsThisSlice // fractionDivider
        chosenScribbleCoordsThisSlice = random.sample(onePixelCoordsThisSlice, numScribblesFromThisSlice)
        #Convert chose coords  in 3D
        chosenScribbleCoordsThisSlice = \
          [ tuple([sliceId] + list(tupleElem)) for  tupleElem in chosenScribbleCoordsThisSlice]
        #print(chosenFGScribbleFromMissedFGCoordsThisSlice)
        for coord in chosenScribbleCoordsThisSlice : resultBinary[coord] = 1 
        #dialate in slice        
        resultSlice = resultBinary[sliceId,...]
        #print('result before dilation ')
        #print(resultSlice)
        resultSlice = \
           scipy.ndimage.binary_dilation(resultSlice,structure=disk(dilation_diam)).astype(resultSlice.dtype)
        #print('result after dilation ')
        #print(resultSlice)
        #But make sure it does not go beyond original binary
        resultSlice = resultSlice * misclassifiedRegion[sliceId,...]
        #print('result after clipping ')
        #print(resultSlice)
        resultBinary[sliceId,...] = resultSlice
    return resultBinary


In [128]:
def chooseScribbleFromDefiniteRegion(definiteRegion, ring_diam):
    """
        definiteRegion : int8 binary volume of definiteRegion with slice as first dimension 
        ring_diam: dimeter of disk : 2, 3, 4 : a diam x diam window is placed to choose scribble
    """
    resultBinary = np.zeros_like(definiteRegion)
    numSlices = definiteRegion.shape[0]
    for sliceId in range(numSlices):
        definiteSlice = definiteRegion[sliceId,...]
        erodedSlice = \
          scipy.ndimage.binary_erosion(definiteSlice,structure=disk(ring_diam)).astype(definiteSlice.dtype)
        scribbleSlice = definiteSlice - erodedSlice
        #print('result after subtraction of erosion ')
        #print(scribbleSlice)
        resultBinary[sliceId,...] = scribbleSlice
    return resultBinary      

In [129]:
#gt = np.zeros((2,16,16), dtype=np.uint8)
#gt[0, 4:11, 6:10]=1
#gt[1, 3:10, 7:12]=1

#pred = np.zeros((2,16,16), dtype=np.uint8)
#pred[0, 3:10, 7:12]=1
#pred[1, 4:11, 6:10]=1

gt = np.zeros((1,20,20), dtype=np.uint8)
gt[0, 7:17, 2:17]=1


pred = np.zeros((1,20,20), dtype=np.uint8)
pred[0, 2:13, 5:13]=1

gt_comp = 1-gt 
pred_comp = 1-pred
gtMinusPred = gt * pred_comp 
predMinusGt = pred * gt_comp

definiteBG = gt_comp # This is also definite background
definiteFG = gt * pred #This is definite FG
fgMissed = gtMinusPred  #  FG missed by classified
bgWrongC = predMinusGt #  BG wrongly classified as FG


# print('gt')
# print(gt)

# print('pred')
# print(pred)

# print('fgMissed')
# print(fgMissed)
fgScribbleFromFGMissed = \
    chooseScribbleFromMissedFGOrWrongCBG(misclassifiedRegion=fgMissed, fractionDivider=4, dilation_diam=1)
# print('fgScribbleFromFGMissed')
# print(fgScribbleFromFGMissed)

# print('bgWrongC')
# print(bgWrongC)
bgScribbleFromBGWrongC = \
    chooseScribbleFromMissedFGOrWrongCBG(misclassifiedRegion=bgWrongC, fractionDivider=4, dilation_diam=1)
# print('bgScribbleFromBGWrongC')
# print(bgScribbleFromBGWrongC)    

# print('definiteFG')
# print(definiteFG)
fgScribbleFromDefiniteFG = chooseScribbleFromDefiniteRegion(definiteRegion=definiteFG, ring_diam=1)
# print('fgScribbleFromDefiniteFG')
# print(fgScribbleFromDefiniteFG)  

print('definiteBG')
print(definiteBG)
bgScribbleFromDefiniteBG = chooseScribbleFromDefiniteRegion(definiteRegion=definiteBG, ring_diam=1)
print('bgScribbleFromDefiniteBG')
print(bgScribbleFromDefiniteBG)


definiteBG
[[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
  [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
  [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
  [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
  [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
  [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
  [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
  [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
  [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
  [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
  [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]]
bgScribbleFromDefiniteBG
[[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [130]:
im = np.random.random([5, 5, 1])
im[:3, :3] += 1.
print('im.dtype ', im.dtype)

seeds = np.zeros([5, 5, 1], dtype=np.uint8)
seeds[:3,0] = 1
seeds[:3,4] = 2

gc = imcut.pycut.ImageGraphCut(im)
gc.set_seeds(seeds)
gc.run()

print(gc.segmentation.squeeze())
print('segmentation shape ', gc.segmentation.shape)

im.dtype  float64
[[0 0 0 1 1]
 [0 0 0 1 1]
 [0 0 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
segmentation shape  (5, 5, 1)
